In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [11]:
# show all columns 
pd.set_option('display.max_columns', None)

In [12]:
# open beer_df.csv
beer_df = pd.read_csv('data/beer_df.csv', low_memory=False)
beer_df.head()

,address,categories,city,country,key,lat,long,brewery_name,phones,postalCode,province,websites,index,brewery_id,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid,review_year,review_month
0,2010 Williams St,Brewery,San Leandro,US,us/ca/sanleandro/2010williamsst,37.711807,-122.177658,21st Amendment Brewery,5105952111,94577,CA,http://21st-amendment.com,1495017,735,2011-03-01 00:49:43,3.5,3.5,4.0,illidurit,American Double / Imperial IPA,3.5,3.5,21 Rock,9.7,66190,2011,3
1,2010 Williams St,Brewery,San Leandro,US,us/ca/sanleandro/2010williamsst,37.711807,-122.177658,21st Amendment Brewery,5105952111,94577,CA,http://21st-amendment.com,1495350,735,2008-12-04 19:03:15,4.0,4.0,4.0,magictrokini,American IPA,3.0,4.0,Harvest Moon,6.4,45648,2008,12
2,2010 Williams St,Brewery,San Leandro,US,us/ca/sanleandro/2010williamsst,37.711807,-122.177658,21st Amendment Brewery,5105952111,94577,CA,http://21st-amendment.com,1495733,735,2010-01-23 20:55:46,4.0,4.0,3.5,HapWifeHapLife,American IPA,4.0,4.0,21st Amendment IPA,7.0,20781,2010,1
3,2010 Williams St,Brewery,San Leandro,US,us/ca/sanleandro/2010williamsst,37.711807,-122.177658,21st Amendment Brewery,5105952111,94577,CA,http://21st-amendment.com,1501253,735,2010-04-08 18:58:54,4.0,3.5,4.5,pwoody11,Belgian Strong Dark Ale,4.0,4.0,Monk's Blood,8.3,52510,2010,4
4,2010 Williams St,Brewery,San Leandro,US,us/ca/sanleandro/2010williamsst,37.711807,-122.177658,21st Amendment Brewery,5105952111,94577,CA,http://21st-amendment.com,1501262,735,2010-03-14 16:30:10,4.0,3.5,4.0,metter98,Belgian Strong Dark Ale,4.0,4.5,Monk's Blood,8.3,52510,2010,3


## Baseline Model

In [13]:
# create baseline model of beer_df with normal_predictor and train test split from surprise using this link: 
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import train_test_split

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(beer_df[['review_profilename', 'beer_name', 'review_overall']], reader)

trainset, testset = train_test_split(data, test_size=.25)

algo = NormalPredictor()
algo.fit(trainset)
predictions = algo.test(testset)

accuracy.rmse(predictions)
accuracy.mae(predictions)

RMSE: 0.8676
MAE:  0.6812


0.6811504568438177

## Running model with Surprise Package

After running a baseline model, we will now run an SVD algoritm to improve our rmse and mae. 

In [14]:
# Running a model with SVD
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import train_test_split

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(beer_df[['review_profilename', 'beer_name', 'review_overall']], reader)

trainset, testset = train_test_split(data, test_size=.25)

algo = SVD()
algo.fit(trainset)
predictions = algo.test(testset)

accuracy.rmse(predictions)
accuracy.mae(predictions)


RMSE: 0.5587
MAE:  0.4172


0.4172179267610751

### Tuned Model running GridSearchCV and RandomizedSearchCV

In [15]:
# run model svd using gridsearchcv
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV

reader = Reader(rating_scale=(1.0, 5.0))
data = Dataset.load_from_df(beer_df[['review_profilename', 'beer_name', 'review_overall']], reader)

param_grid = {'n_factors': [50, 100, 150], 'n_epochs': [20, 40, 60], 'lr_all': [0.002, 0.005, 0.008], 'reg_all': [0.4, 0.6, 0.8]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])


0.5616435071805842
{'n_factors': 50, 'n_epochs': 60, 'lr_all': 0.002, 'reg_all': 0.4}


In [16]:
# run model svd using randomizedsearchcv
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import RandomizedSearchCV

reader = Reader(rating_scale=(1.0, 5.0))
data = Dataset.load_from_df(beer_df[['review_profilename', 'beer_name', 'review_overall']], reader)

param_grid = {'n_factors': [50, 100, 150], 'n_epochs': [20, 40, 60], 'lr_all': [0.002, 0.005, 0.008], 'reg_all': [0.4, 0.6, 0.8]}
gs = RandomizedSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])


0.5618820518744397
{'n_factors': 150, 'n_epochs': 40, 'lr_all': 0.005, 'reg_all': 0.4}


## Running model with SciKit-Learn

In [35]:
# run model using sci-kit learn and truncatedSVD


### Using the best model to create recommendation system